In [1]:
import pandas as pd
import numpy as np
import datetime
from holidays_jp import CountryHolidays
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import pickle
import streamlit as st
#  import altair as alt
import matplotlib.pyplot as plt
import os



In [2]:
# @st.cache
def load_data():
    # get data in memory
    infile = open('assets/disaster_daily_edits.pickle','rb') 
    daily_edits = pickle.load(infile)
    infile.close()

    # get japanese holidays in date range
    holidays2019 = CountryHolidays.get('JP', 2019)
    holidays2020 = CountryHolidays.get('JP', 2020)
    holidays2021 = CountryHolidays.get('JP', 2021)
    holiday_list = [holidays2019, holidays2020, holidays2021]
    holidays = pd.concat(map(pd.DataFrame, holiday_list), axis='rows').set_index(0)        
    holidays.index = holidays.index.tz_localize('Japan')
    holidays = holidays.resample('D').asfreq(' ').rename(columns={1:'holiday_name'})
    holidays['holiday'] = holidays.holiday_name.map(lambda x: int(x != ' '))

    # prep data
    disasters_english = {'火山災害':'VolcanicDisaster', 
                        '熱帯低気圧':'TropicalCyclones', 
                        '雪害':'SnowDamage', 
                        '地震':'Earthquake', 
                        '津波':'Tsunami'}       
    ts = daily_edits.copy()
    ts = ts.rename(columns=disasters_english)
    ts['mtwtf'] = ts.index.dayofweek.isin([0,1,2,3,4]).astype(int)
    ts['sat'] = ts.index.dayofweek.isin([5]).astype(int)
    ts['sun'] = ts.index.dayofweek.isin([5]).astype(int)
    ts['holiday'] = holidays['holiday']
    ts['holiday_on_weekday'] = ts[['holiday', 'mtwtf']].all(axis='columns').astype(int)
    ts = ts[ts.columns.difference(['holiday'])]
    ts = ts.iloc[1:-1]    
    calendar_cols = ['holiday_on_weekday','mtwtf','sat', 'sun',]
    
    ts_averages = {i:ts[i].mean() for i in ts}
    
    return ts, list(disasters_english.values()), calendar_cols

# @st.cache
# def instantiate_results(counter):
#     results = []
#     return results



In [3]:
os.chdir('../')

In [4]:
ts, target_names_default, calendar_cols = load_data()
p_AR_parameter = 3
moving_average = 10

In [5]:
def model_prep(target_names,ts,p_AR_parameter,moving_average):
    ts_lags = ts.copy()

    lag_vars, μ_vars = [], []
    for i in target_names:
        for j in list(range(1,p_AR_parameter+1)):
            ts_lags[f"{i}_l{j}"] = ts_lags[i].shift(j)
            lag_vars.append(f"{i}_l{j}")
        for j in [moving_average]:
            ts_lags[f"{i}_μ{j}"] = ts_lags[i].rolling(window=j, closed="left").mean()
            μ_vars.append(f"{i}_μ{j}")
            
    ts_lags = ts_lags.dropna()

    XX = ts_lags[ts_lags.columns.difference(target_names)]
    YY = ts_lags[target_names]

    return XX, YY


In [6]:
def feature_columns(XX,target_names,calendar_cols):
    Xcols = {}
    for i in target_names:
        Xcols[i] = XX.columns[XX.columns.str.startswith(i)].tolist() + calendar_cols
    return Xcols

In [7]:
target_names = target_names_default
XX,YY = model_prep(target_names,ts,p_AR_parameter,moving_average)

2022-03-16 12:34:58.140 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [8]:
def model_fit(ts,p_AR_parameter,moving_average,target_names,calendar_cols):
    XX,YY = model_prep(target_names,ts,p_AR_parameter,moving_average)

    start_tr = 0
    end_tr = 730
    end_vl = XX.shape[0]
    XXtr, YYtr = XX.copy().iloc[start_tr:end_tr], YY.iloc[start_tr:end_tr]
    XXvl, YYvl = XX.copy().iloc[end_tr:end_vl], YY.iloc[end_tr:end_vl]

    Xcols = feature_columns(XX,target_names,calendar_cols)

    ycols = {i:i for i in target_names}

    Xtr, Xvl = {}, {}
    ytr, yvl = {}, {}
    for diz in target_names:
        Xtr[diz] = XXtr[Xcols[diz]]
        Xvl[diz] = XXvl[Xcols[diz]]
        ytr[diz] = YYtr[ycols[diz]]
        yvl[diz] = YYvl[ycols[diz]]

    ri, gs_ri, scores = {}, {}, {}
    for diz in Xtr:
        ri[diz] = Ridge()
        gs_ri[diz] = GridSearchCV(ri[diz], {'alpha': [-100,-10,0,10,100]})
        gs_ri[diz].fit(Xtr[diz],ytr[diz])
        scores[diz] = gs_ri[diz].score(Xvl[diz],yvl[diz])
        
    return gs_ri,scores,XX,YY,Xcols

In [9]:
gs_ri,scores,XX,YY,Xcols = (
        model_fit(ts,p_AR_parameter,moving_average,target_names,calendar_cols)
    )

/home/bhrdwj/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.92027e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/bhrdwj/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=3.40112e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/bhrdwj/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.55801e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/bhrdwj/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=1.23457e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/bhrdwj

In [10]:
pd.DataFrame(scores, index=['R²'])

,VolcanicDisaster,TropicalCyclones,SnowDamage,Earthquake,Tsunami
R²,-0.016108,-0.08988,0.019656,0.392525,0.410899


In [11]:
i = 'Earthquake'
(pd.DataFrame({
    'feat': Xcols[i],
    'coeffs': gs_ri[i].best_estimator_.coef_.round(3).tolist(), 
    'mean': XX[Xcols[i]].mean().round(3).tolist(),
    'std': XX[Xcols[i]].std().round(3).values.tolist()
})
 .set_index('feat')
 .assign(coeff_x_std=lambda x: round(x['coeffs'] * x['std'],3))
 .sort_values(by='coeff_x_std', key=abs, ascending=False)
 
)

,coeffs,mean,std,coeff_x_std
feat,,,,
Earthquake_l1,0.320,304.530,74.431,23.818
Earthquake_μ10,0.395,304.125,54.399,21.488
mtwtf,-30.686,0.714,0.452,-13.870
holiday_on_weekday,52.360,0.064,0.244,12.776
sat,-21.202,0.143,0.350,-7.421
sun,14.192,0.143,0.350,4.967
Earthquake_l3,0.060,304.350,74.432,4.466
Earthquake_l2,0.016,304.455,74.432,1.191


In [12]:
coeffs = {}

for diz in gs_ri:
    coeffs[diz] = pd.Series(
        gs_ri[diz].best_estimator_.coef_, 
        index=[1,2,3,4,5,6,7,8], 
        name=diz)
    print(Xcols[diz])
pd.DataFrame(coeffs)

['VolcanicDisaster_l1', 'VolcanicDisaster_l2', 'VolcanicDisaster_l3', 'VolcanicDisaster_μ10', 'holiday_on_weekday', 'mtwtf', 'sat', 'sun']
['TropicalCyclones_l1', 'TropicalCyclones_l2', 'TropicalCyclones_l3', 'TropicalCyclones_μ10', 'holiday_on_weekday', 'mtwtf', 'sat', 'sun']
['SnowDamage_l1', 'SnowDamage_l2', 'SnowDamage_l3', 'SnowDamage_μ10', 'holiday_on_weekday', 'mtwtf', 'sat', 'sun']
['Earthquake_l1', 'Earthquake_l2', 'Earthquake_l3', 'Earthquake_μ10', 'holiday_on_weekday', 'mtwtf', 'sat', 'sun']
['Tsunami_l1', 'Tsunami_l2', 'Tsunami_l3', 'Tsunami_μ10', 'holiday_on_weekday', 'mtwtf', 'sat', 'sun']


,VolcanicDisaster,TropicalCyclones,SnowDamage,Earthquake,Tsunami
1,0.033047,0.506460,0.027527,0.320354,0.318431
2,-0.008845,0.224406,0.005317,0.016471,0.015659
3,0.014276,-0.010990,-0.012031,0.060386,0.058823
4,-0.007490,0.201471,0.083992,0.395484,0.432780
5,0.307229,-0.391044,0.010492,52.360000,34.733619
6,0.121042,-1.112377,-0.027786,-30.686063,-24.697359
7,-0.019173,-0.038954,0.006809,-21.201837,-0.826891
8,-0.019173,-0.038954,0.006809,14.192218,-0.826891


In [13]:
# def grid_search():
#     # load data - should only happen once
#     ts, target_names_default, calendar_cols = load_data()
#     st.title("Manual Grid Search")
#     # user input
#     with st.form("inputs"):

#         target_names = st.multiselect(
#             "Select your target",
#             target_names_default,
#             default = target_names_default
#         )
#         p_AR_parameter = st.select_slider("select # of lags",
#         options=range(1,6),value=3
#         )
#         moving_average = st.select_slider(
#             "select # of days for moving average",
#             options=range(7,14)
#         )
        
#         submitted = st.form_submit_button("Compute!")
    
#     gs_ri,scores,XX,YY,Xcols = model_fit(ts,p_AR_parameter,moving_average,target_names,calendar_cols)
    
#     result_dict = {}
#     result_dict['target_names'] = target_names
#     result_dict['p_AR_parameter'] = p_AR_parameter
#     result_dict['moving_average'] = moving_average

#     # for diz in target_names:
#     #     result_dict[diz+'_score'] = scores[diz]
#     #     st.write(f"{diz}: {scores[diz]:.2f}")
#     round_scores = {i:round(scores[i], 3) for i in scores}
#     st.write("Results")
#     st.dataframe(pd.DataFrame(scores, index=['R²']))
#     st.write("Actual and Predicted Wikipedia Edits by Category")
#     model_plot(result_dict, gs_ri,target_names,XX,YY,Xcols)